In [15]:
import os
import re
from typing import Optional
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, field_validator
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
# OPENAI_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv()

True

In [16]:
# 뉴스 기사 구조를 정의하는 Pydantic 모델입니다.
class NewsArticle(BaseModel):
    """뉴스 기사의 구조화된 정보를 담는 모델"""

    press: str = Field(description="언론사 이름")
    news_link: str = Field(description="뉴스 기사 링크")
    title: str = Field(description="뉴스 기사의 제목")
    subtitle: str = Field(description="뉴스 기사의 소제목 (있을 경우)")
    input_time: str = Field(description="뉴스 입력 시간")
    modified_time: Optional[str] = Field(
        description="뉴스 수정 시간 (없을 수 있음)", default=None
    )
    reporter_name: str = Field(description="취재 기자 이름")
    reporter_email: Optional[str] = Field(
        description="취재 기자 이메일 (없을 수 있음)", default=None
    )
    summary: str = Field(description="뉴스 기사 내용의 핵심 요약 (100자 이내)")
    subscribers: int = Field(description="뉴스 기사 구독자 수")
    supporters: int = Field(description="뉴스 기사 응원자 수")

    @field_validator("subscribers", "supporters", mode="before")
    def parse_number_from_string(cls, v):
        """숫자 문자열을 정수로 변환"""
        if isinstance(v, str):
            # '2,227' 같은 형식을 정수로 변환
            return int(v.replace(",", "").strip())
        return v

In [17]:
# PydanticOutputParser를 초기화합니다.
parser = PydanticOutputParser(pydantic_object=NewsArticle)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "뉴스 기사의 구조화된 정보를 담는 모델", "properties": {"press": {"description": "언론사 이름", "title": "Press", "type": "string"}, "news_link": {"description": "뉴스 기사 링크", "title": "News Link", "type": "string"}, "title": {"description": "뉴스 기사의 제목", "title": "Title", "type": "string"}, "subtitle": {"description": "뉴스 기사의 소제목 (있을 경우)", "title": "Subtitle", "type": "string"}, "input_time": {"description": "뉴스 입력 시간", "title": "Input Time", "type": "string"}, "modified_time": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "descr

In [18]:
# 프롬프트 템플릿을 생성합니다.
prompt_template = """
당신은 뉴스 기사를 읽고 아래에 주어진 JSON 형식에 맞게 정보를 추출하여 요약하는 전문가입니다.
주어진 형식에 맞게 JSON을 출력해야 합니다.

{format_instructions}

다음은 요약할 뉴스 기사입니다:
{news_article}
"""

# PromptTemplate 객체를 생성합니다.
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["news_article"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [6]:
# 사용자에게 받은 뉴스 기사 텍스트입니다.
example_news_article = """
뉴스기사링크 : https://n.news.naver.com/mnews/article/008/0005237750
머니투데이
中 저가 공세 맞불...삼성, 10만원대 '갤버즈3 FE' 내달 출격
입력2025.08.19. 오전 11:00  수정2025.08.19. 오후 2:06
김승한 기자

(종합)
지난해 출시 '갤버즈3' 하위 파생 모델
韓 출고가 10만원 중반 예상, 9월 출시

삼성전자가 보급형 무선이어폰 신제품 '갤럭시버즈3 FE(팬에디션)'를 내달 선보인다. 합리적인 가격에 주요 기능을 담아 실속형 소비자층을 겨냥하고, 가성비를 앞세운 중국 업체를 견제하기 위한 전략으로 해석된다.

19일 삼성전자에 따르면 갤럭시버즈3 FE는 내달 5일 미국과 유럽을 시작으로 순차 출시된다. 한국도 내달 중 출시된다. 미국 출고가는 149.99달러(약 20만8000원)로, 지난해 7월 나온 '갤럭시버즈3'(179.99달러)보다 30달러(약 4만원) 저렴하다. 반면 전작 격인 '갤럭시버즈 FE'(99.99달러)보다는 50달러(약 7만원) 비싸다. 한국 출고가는 10만원 중반대로 예상되며 출시 시점에 맞춰 확정될 예정이다. 색상은 블랙과 그레이 두 가지다.

갤럭시버즈3 FE는 갤럭시버즈3의 주요 기능과 디자인을 유지하면서 가격을 낮춘 하위 파생 모델이다. 삼성은 특히 향상된 액티브 노이즈 캔슬링(ANC) 기능을 강조하며 몰입감 있는 청취 경험을 제공한다고 설명했다.

또 마이크 위치를 최적화하고 머신러닝 기반 알고리즘을 적용해 수음 품질을 높였으며, 통화 환경도 개선했다. 블레이드를 쓸거나 집는 등 직관적인 제스처로 볼륨 조절 등 다양한 기능을 사용할 수 있고, 케이스의 페어링 버튼을 통해 스마트폰·태블릿·PC 등 갤럭시 기기와 간편하게 연결할 수 있다. '오토 스위치' 기능으로 기기 간 자동 전환도 지원한다.

AI(인공지능) 기능도 강화됐다. '헤이 구글' 명령어나 블레이드 길게 누르기로 삼성의 AI 비서 '제미나이'를 호출할 수 있으며, 일정 확인이나 메시지 조회 등 스마트폰 없이도 음성 대화가 가능하다. 실시간 통역 기능도 지원해 외국어 강의나 해외 여행 등에서 활용도를 높였다.

업계에선 삼성이 이번 보급형 신제품 출시를 통해 가성비 제품으로 글로벌 시장을 공략 중인 중국 업체와의 격차를 벌리려는 전략으로 본다. 샤오미, 화웨이 등은 최근 10만원대 안팎의 무선이어폰을 내세워 신흥국뿐 아니라 유럽·동남아 시장까지 영향력을 넓히고 있다. 이들이 가격 경쟁력을 앞세워 빠르게 점유율을 늘려가고 있는 만큼, 삼성도 보급형 라인업 강화로 대응하지 않으면 중저가 시장에서 입지가 더 줄어들 수 있다는 우려가 제기된다.

글로벌 시장에서 삼성의 입지는 다소 약화됐다. 시장조사업체 카날리스에 따르면 올해 1분기 전 세계 무선이어폰 시장에서 삼성 점유율은 7.1%로 전년 동기 대비 0.7%포인트 감소했다. 같은 기간 1위인 애플의 점유율도 23.3%로 1.1%포인트 줄었다. 반면 중국 샤오미(11.5%)와 화웨이(6.0%)는 점유율을 확대하며 삼성과 경쟁하고 있다.

업계 한 관계자는 "무선이어폰 시장이 성숙기에 접어들면서 프리미엄만으로는 점유율을 유지하기 어렵다"며 "삼성이 FE 모델을 통해 중저가 수요층을 흡수할 수 있다면 애플과 중국 업체 사이에서 균형을 잡는 발판이 될 수 있을 것"이라고 말했다.
김승한 기자 (winone@mt.co.kr)

구독자 2,227  
응원수 2,121
"""

In [20]:
# 모델을 초기화합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

# 프롬프트, 모델, 파서를 체인으로 연결합니다.
chain = prompt | model | parser

# 체인을 실행하고 요약 결과를 얻습니다.
summary_result = chain.invoke({"news_article": example_news_article})

In [21]:
# 결과 출력
print("--- 요약 결과 ---")
print(f"언론사: {summary_result.press}")
print(f"뉴스 링크: {summary_result.news_link}")
print(f"제목: {summary_result.title}")
print(f"소제목: {summary_result.subtitle}")
print(f"입력 시간: {summary_result.input_time}")
print(f"수정 시간: {summary_result.modified_time}")
print(f"취재 기자: {summary_result.reporter_name}")
print(f"기자 이메일: {summary_result.reporter_email}")
print(f"뉴스 요약: {summary_result.summary}")
print(f"구독자 수: {summary_result.subscribers}")
print(f"응원자 수: {summary_result.supporters}")
print("\n" + "=" * 50 + "\n")
print("--- 생성된 객체 정보 ---")
print(f"타입: {type(summary_result)}")
print(f"내용: {summary_result}")

--- 요약 결과 ---
언론사: 머니투데이
뉴스 링크: https://n.news.naver.com/mnews/article/008/0005237750
제목: 中 저가 공세 맞불...삼성, 10만원대 '갤버즈3 FE' 내달 출격
소제목: 지난해 출시 '갤버즈3' 하위 파생 모델
韓 출고가 10만원 중반 예상, 9월 출시
입력 시간: 2025.08.19. 오전 11:00
수정 시간: 2025.08.19. 오후 2:06
취재 기자: 김승한
기자 이메일: winone@mt.co.kr
뉴스 요약: 삼성전자가 가성비를 앞세운 '갤럭시버즈3 FE'를 9월 출시해 중국 업체와 경쟁한다.
구독자 수: 2227
응원자 수: 2121


--- 생성된 객체 정보 ---
타입: <class '__main__.NewsArticle'>
내용: press='머니투데이' news_link='https://n.news.naver.com/mnews/article/008/0005237750' title="中 저가 공세 맞불...삼성, 10만원대 '갤버즈3 FE' 내달 출격" subtitle="지난해 출시 '갤버즈3' 하위 파생 모델\n韓 출고가 10만원 중반 예상, 9월 출시" input_time='2025.08.19. 오전 11:00' modified_time='2025.08.19. 오후 2:06' reporter_name='김승한' reporter_email='winone@mt.co.kr' summary="삼성전자가 가성비를 앞세운 '갤럭시버즈3 FE'를 9월 출시해 중국 업체와 경쟁한다." subscribers=2227 supporters=2121
